<a href="https://colab.research.google.com/github/Galeano-Garcia-GS/Synergistic-SCFA-production-by-Fiber-Mixture/blob/main/Synergy_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Synergy calculation
Find [metadata](https://docs.google.com/spreadsheets/d/1WkbV79p_IPtRoRTBd_aIz_68Df5ms7uy24yDXEpAAIk/edit?usp=sharing) file here.

In [ ]:
#@title Dataframe

install_dependencies = False  #@param {type:"boolean"}
excel_file_path = "/content/Metadata.xlsx"  #@param {type:"string"}
sheet_name = "Full"  #@param {type:"string"}
columns_csv = "Individual, Replicate, Fiber, Acetate_(mM), Expected_acetate_(mM), Butyrate_(mM), Expected_butyrate_(mM), Propionate_(mM), Expected_propionate_(mM), Total_SCFA_(mM), Expected_total_SCFA_(mM)"  #@param {type:"string"}
enable_filtering = True  #@param {type:"boolean"}

output_directory = "/content"  #@param {type:"string"}
output_filename = "Metadata_V1.xlsx"  #@param {type:"string"}
create_directory_if_missing = True  #@param {type:"boolean"}
save_format = "Excel (.xlsx)"  #@param ["Excel (.xlsx)", "CSV (.csv)"]
include_index = False  #@param {type:"boolean"}

import sys, subprocess, os
if install_dependencies:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "pandas", "openpyxl"])

import pandas as pd
from IPython.display import display, HTML
import ipywidgets as widgets

if not os.path.exists(excel_file_path):
    raise FileNotFoundError(f"File not found: {excel_file_path}")

raw_df = pd.read_excel(excel_file_path, sheet_name=sheet_name, header=None)
headers = raw_df.iloc[0].tolist()
types = raw_df.iloc[1].tolist()
data = raw_df.iloc[2:].reset_index(drop=True)
data.columns = headers

selected_columns = [c.strip() for c in columns_csv.split(",") if c.strip()] or headers
missing = [c for c in selected_columns if c not in headers]
if missing:
    raise KeyError(f"Columns not found: {missing}\nAvailable: {headers}")

df_selected = data[selected_columns].copy()
display(HTML("<h4>Preview of selected columns</h4>"))
display(df_selected.head())
print(f"Shape: {df_selected.shape}")

type_map = dict(zip(headers, types))
categorical_cols = [c for c in selected_columns if str(type_map.get(c, "")).lower() == "categorical"]

filter_widgets = {}
clear_buttons = {}

if enable_filtering and categorical_cols:
    display(HTML("<h4>Filters for categorical columns</h4>"))
    cols_ui = []
    for col in categorical_cols:
        unique_vals = df_selected[col].dropna().unique().tolist()
        has_nans = df_selected[col].isna().any()

        ms = widgets.SelectMultiple(options=unique_vals, description=col, layout=widgets.Layout(width="60%"))
        nan_toggle = widgets.Checkbox(value=False, description="Include missing") if has_nans else None
        clear_btn = widgets.Button(description="Clear", button_style="warning", icon="x")

        def make_clear_handler(ms_widget, nan_widget):
            def _handler(_):
                ms_widget.value = ()
                if nan_widget is not None:
                    nan_widget.value = False
            return _handler

        clear_btn.on_click(make_clear_handler(ms, nan_toggle))
        row = widgets.HBox([ms, widgets.VBox(([nan_toggle] if nan_toggle else [])), clear_btn])
        cols_ui.append(row)

        filter_widgets[col] = {"widget": ms, "nan": nan_toggle}
        clear_buttons[col] = clear_btn

    display(widgets.VBox(cols_ui))

apply_button = widgets.Button(description="Apply Filters & Save", button_style="primary", icon="check")
clear_all_button = widgets.Button(description="Clear All Filters", icon="trash")
buttons_bar = widgets.HBox([apply_button, clear_all_button])

output_area = widgets.Output()
display(buttons_bar, output_area)

def clear_all(_):
    for parts in filter_widgets.values():
        parts["widget"].value = ()
        if parts["nan"] is not None:
            parts["nan"].value = False

def apply_filters_and_save(_):
    with output_area:
        output_area.clear_output()
        df_filtered = df_selected.copy()
        if enable_filtering and filter_widgets:
            for col, parts in filter_widgets.items():
                chosen = list(parts["widget"].value)
                include_nan = parts["nan"].value if parts["nan"] else False
                if chosen or include_nan:
                    mask = df_filtered[col].isin(chosen)
                    if include_nan: mask |= df_filtered[col].isna()
                    df_filtered = df_filtered[mask]
        display(HTML("<h4>Final DataFrame</h4>"))
        display(df_filtered)
        print(f"Final shape: {df_filtered.shape}")
        if create_directory_if_missing and not os.path.exists(output_directory):
            os.makedirs(output_directory, exist_ok=True)
        out_path = os.path.join(output_directory, output_filename)
        if save_format.startswith("Excel"):
            if not out_path.lower().endswith(".xlsx"): out_path += ".xlsx"
            df_filtered.to_excel(out_path, index=include_index)
        else:
            if not out_path.lower().endswith(".csv"): out_path += ".csv"
            df_filtered.to_csv(out_path, index=include_index)
        display(HTML(f"<b>Saved to:</b> <code>{out_path}</code>"))

clear_all_button.on_click(clear_all)
apply_button.on_click(apply_filters_and_save)

,Individual,Replicate,Fiber,Acetate_(mM),Expected_acetate_(mM),Butyrate_(mM),Expected_butyrate_(mM),Propionate_(mM),Expected_propionate_(mM),Total_SCFA_(mM),Expected_total_SCFA_(mM)
0,C01,1,FOS,4.264,NaN,2.285,NaN,3.466,NaN,10.015,NaN
1,C01,2,FOS,3.717,NaN,1.985,NaN,3.032,NaN,8.734,NaN
2,C01,1,β-Glucan,6.708,NaN,2.039,NaN,3.079,NaN,11.826,NaN
3,C01,2,β-Glucan,2.425,NaN,2.021,NaN,2.983,NaN,7.43,NaN
4,C01,1,Pectin,7.166,NaN,1.373,NaN,2.314,NaN,10.853,NaN


Shape: (408, 11)


Output()

In [ ]:
#@title Calculating synergy
import pandas as pd
import numpy as np

# ---- Load data ----
metadata_file_path = "Metadata_V1.xlsx"  #@param {type:"string"}
df = pd.read_excel(metadata_file_path)

# ---- Column handles ----
observed_column = "Acetate_(mM)"  #@param ["Acetate_(mM)", "Butyrate_(mM)", "Propionate_(mM)", "Total_SCFA_(mM)"]
expected_column = "Expected_acetate_(mM)"  #@param ["Expected_acetate_(mM)", "Expected_butyrate_(mM)", "Expected_propionate_(mM)", "Expected_total_SCFA_(mM)"]

# ---- Tunable parameters (Colab form fields) ----
measurement_error = 0.5  #@param {type:"slider", min:0.0, max:1.0, step:0.01}
minimal_detection_limit = 1  #@param {type:"slider", min:0.0, max:1.0, step:0.01}

# Convert to numeric safely
for c in [observed_column, expected_column]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# ---- Gating rules ----
diff = df[observed_column].sub(df[expected_column]).abs()
gate_mdl = (df[observed_column] >= minimal_detection_limit) & (df[expected_column] >= minimal_detection_limit)
gate_me = (diff > measurement_error)  # keep your original strict '>' rule
gate_nonzero = df[expected_column] != 0
mask_valid = gate_mdl & gate_me & gate_nonzero

# ---- Prepare output ----
synergy_col = f"Synergy_in_{observed_column}" + "_(%)"
out = df[["Individual", "Replicate", observed_column, expected_column]].copy()

# Calculate synergy only where valid; otherwise NaN for now
out[synergy_col] = np.where(
    mask_valid,
    ((df[observed_column] - df[expected_column]) / df[expected_column]) * 100,
    np.nan
)

synergy_category = f"{observed_column}" + "_synergy_category"

# Create category column with object dtype first
out[synergy_category] = pd.Series(index=out.index, dtype="object")
out.loc[out[synergy_col] >= 0, synergy_category] = "Positive"
out.loc[out[synergy_col] < 0, synergy_category] = "Negative"

# ---- Apply requested exclusions behavior ----
# If excluded ONLY due to measurement_error -> set synergy to 0
fail_me = ~gate_me
fail_mdl = ~gate_mdl
me_only = (~mask_valid) & fail_me & (~fail_mdl) & gate_nonzero
out.loc[me_only, synergy_col] = 0.0

# Reason of exclusion column
reason_col = f"Reason_of_exclusion_{synergy_category}"
out[reason_col] = pd.Series(index=out.index, dtype="object")
out.loc[fail_mdl & fail_me, reason_col] = "minimal_detection_limit and measurement_error"
out.loc[fail_mdl & ~fail_me, reason_col] = "minimal_detection_limit"
out.loc[(~fail_mdl) & fail_me, reason_col] = "measurement_error"
# rows that are valid remain NaN (blank in Excel)

# Round synergy for readability
out[synergy_col] = out[synergy_col].round(3)

# NEW: If synergy == 0, set category to "No_synergy"
out.loc[out[synergy_col] == 0, synergy_category] = "No_synergy"

# Final column order (original order + reason column at the end)
out = out[["Individual", "Replicate", observed_column, expected_column, synergy_col, synergy_category, reason_col]]

# ---- Save & display ----
save_path = f"/content/{observed_column}_synergy_results.xlsx"
out.to_excel(save_path, index=False)

print(f"Input rows: {len(df)}")
print(f"Rows with synergy calculated: {mask_valid.sum()}")
print(f"Saved to: {save_path}")
out.head(10)

Input rows: 68
Rows with synergy calculated: 21
Saved to: /content/Propionate_(mM)_synergy_results.xlsx


,Individual,Replicate,Propionate_(mM),Expected_propionate_(mM),Synergy_in_Propionate_(mM)_(%),Propionate_(mM)_synergy_category,Reason_of_exclusion_Propionate_(mM)_synergy_category
0,C01,1,3.433,2.949,0.00,No_synergy,measurement_error
1,C01,2,3.848,3.091,24.49,Positive,NaN
2,C02,1,0.483,0.462,NaN,NaN,minimal_detection_limit and measurement_error
3,C02,2,0.501,0.444,NaN,NaN,minimal_detection_limit and measurement_error
4,C03,1,1.601,1.304,0.00,No_synergy,measurement_error
5,C03,2,1.525,1.202,0.00,No_synergy,measurement_error
6,C04,1,1.210,1.345,0.00,No_synergy,measurement_error
7,C04,2,1.450,1.350,0.00,No_synergy,measurement_error
8,C05,1,NaN,NaN,NaN,NaN,minimal_detection_limit and measurement_error
9,C05,2,NaN,NaN,NaN,NaN,minimal_detection_limit and measurement_error
